In [2]:
import pandas as pd
import os
import glob
import numpy as np


In [233]:
ndjson_files = glob.glob(os.path.join(os.getcwd() + r"\\data", "*ndjson.gz"))
csv_files = glob.glob(os.path.join(os.getcwd() + r"\\data", "*csv.gz"))
print(f"json files: {ndjson_files}")
print(f"csv files: {csv_files}")

json files: ['d:\\Dataset\\Parkster\\notebook\\\\data\\events_a.ndjson.gz', 'd:\\Dataset\\Parkster\\notebook\\\\data\\events_b.ndjson.gz', 'd:\\Dataset\\Parkster\\notebook\\\\data\\events_before_test.ndjson.gz']
csv files: ['d:\\Dataset\\Parkster\\notebook\\\\data\\extra_info_a.csv.gz', 'd:\\Dataset\\Parkster\\notebook\\\\data\\extra_info_b.csv.gz', 'd:\\Dataset\\Parkster\\notebook\\\\data\\extra_info_before_test.csv.gz']


In [234]:
csv_files[0][-3:]

'.gz'

In [235]:
def filename (path_name):
    # function that creates a new folder called "fixed" where to save the fixed files
    dir_path, filename = os.path.split(path_name)
    
    new_dir_path = os.path.join(dir_path, 'fixed')
    
    if not os.path.exists(new_dir_path):
        os.mkdir(new_dir_path)
    
    new_file_path = os.path.join(new_dir_path, filename[:-3])
    
    if new_file_path[-3:] != 'csv':
        new_file_path = new_file_path[:-7] + ".csv"
    
    return new_file_path
    

In [236]:
def csv_cleaner(csv_files):
    for i in csv_files:
        # load csv-file, replace double comma with single comma, and split column into 4 columns.
        df = pd.read_csv(i, compression='gzip',sep="\t", header=None)
        df = df.replace(',,', ',')
        df[['cl1', 'cl2', 'cl3', 'cl4']] = df.iloc[:, 0].str.split(',', expand=True)
        
        df = df.fillna('')
        
        # combine column cl3 and cl4 (2 gender columns) into 1
        df['cl5'] = df['cl3'] + df['cl4']
        
        # only keep relevant columns user_id, country and gender (the combined)
        df = df.iloc[:, [1,2,5]]
        
        # while loading the csv-file we didnt use the header, now we create header based on value in row 1
        df.columns = df.iloc[0]
        df = df.drop(df.index[0])

        # error in country column, mix of upper and lower case, and Sweden and Sverige both exist in the column
        df = df.replace('', np.nan)
        df['country'] = df['country'].str.lower()
        df['country'] = df['country'].replace('sverige', 'sweden')
    
        print(df.info())
        print("")
        
        new_file = filename(i)
        df.to_csv(new_file)
        

In [237]:
def ndjson_cleaner(ndjson_files):
    for i in ndjson_files:
        df = pd.read_json(i, lines=True)
        new_file = filename (i)
        df.to_csv(new_file)
        print(df.info())
        print("")

In [238]:
csv_cleaner(csv_files)
ndjson_cleaner(ndjson_files)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 1 to 888
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  888 non-null    object
 1   country  812 non-null    object
 2   gender   816 non-null    object
dtypes: object(3)
memory usage: 20.9+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893 entries, 1 to 893
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  893 non-null    object
 1   country  804 non-null    object
 2   gender   805 non-null    object
dtypes: object(3)
memory usage: 21.1+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8900 entries, 1 to 8900
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  8900 non-null   object
 1   country  8131 non-null   object
 2   gender   8071 non-null   object
dtypes: object(3)
memory usage: 208.7